In [1]:
%matplotlib inline

import numpy as np

from matplotlib import rc
import matplotlib.pyplot as plt
import scipy.stats as st

import sys
sys.path.append("/home/joshua/Documents/Uni/Year4/dissertation/catkin_ws/src/linefollow_gazebo/scripts")


import helper as h
from CameraModel import Camera
from RoadModel import Road, NormalProbabilityDist
from Path import Path

In [2]:
def get_predicted_error_radius(camera_height,
                               ground_distance, 
                               pixel_area,
                               sigma_pos=0.03/2.0,
                               sigma_orient_deg=0.5/2,
                               alg_error=4.0, # note: changed
                               verbose=False,
                               orientation_percent=0.9,
                               pos_percent=0.9,
                               alg_percent=0.9,
                               use_err_xy=True,
                               use_err_z=True,
                               use_err_orient=True,
                               use_err_pixel=True
                              ):
    # TODO this is wrong implementation! Doesn't matter though really as long as use is consistent
    # but really it should be -1*norm.ppf((1-percent)/2.0)
    # as it is is an underestimate

    stddevs_orientation = st.norm.ppf(orientation_percent)
    stddevs_position = st.norm.ppf(pos_percent)
    stddevs_alg = st.norm.ppf(alg_percent)
    
    
    # NOT invariant between pixels
    z_error = (ground_distance/camera_height)*stddevs_position*sigma_pos if use_err_z else 0
    
    # NOT invariant between pixels
    if use_err_orient:
        t = stddevs_orientation*np.deg2rad(sigma_orient_deg)
        alpha = np.arctan2(ground_distance, camera_height)
        beta = alpha + t
        if beta > np.pi/2: # causes tan to roll over into negatives...
            orientation_error = 9999999999
        else:
            orientation_error = np.tan(beta)*camera_height - ground_distance
    else:
        orientation_error = 0
    
    pixel_error = np.sqrt(2*pixel_area) if use_err_pixel else 0
    
#     print("Z error: {0}".format(z_error))
#     print("orien error: {0}".format(orientation_error))
#     print("pixel_error: {0}".format(pixel_error))
    total = z_error + orientation_error + (alg_error+1)/2.0 * stddevs_alg * pixel_error
    return total, z_error, orientation_error, alg_error*pixel_error

In [3]:
def error_radius_across_pixels(camera):
    
    image = np.zeros(shape=(camera.R_y, camera.R_x+1)) # add one because using pixel as bottom left corner! Symmetry matters here
    camera_x, camera_y, camera_z = camera.position
    camera_ground_pos = np.array([camera_x, camera_y, 0.0])

    for (y,x), _ in np.ndenumerate(image):

        ground_point = camera.pixel_to_plane(x, y)
        pixel_area = camera.plane_area_of_pixel(x,y)

        if ground_point is None or pixel_area is None:
            image[y, x] = 999999999999
            continue

        ground_distance = np.linalg.norm(ground_point - camera_ground_pos)

        image[y, x] = get_predicted_error_radius(camera_z, ground_distance, pixel_area)[0]

        return image
        

In [4]:
# basically follows the pseudocode from wikipedia for continuous hillclimbing

def hillclimb(score_function, parameter_ranges, parameter_initializations=None, finish_epsilon=0.1, parameter_step_fraction=0.05, acceleration=1.2):
    """ Takes a scoring function, a set of parameter ranges, parameter initializations if desired, the finishing epsilon, and the fraction of the parameter range that one step takes initially    
    """
    
    params = []
    # if no initializations, choose one randomly
    if parameter_initializations is None:
        for param_range in parameter_ranges:
            params.append(np.random.uniform(*param_range))
    else:
        params = parameter_initializations
    params = np.array(params)
    
    params_ranges = np.array([upper - lower for (lower, upper) in parameter_ranges])
    params_step = params_ranges * parameter_step_fraction
    
    movements = np.array([-acceleration, -1.0/acceleration, 0, 1.0/acceleration, acceleration])
    
    
    score = score_function(params)
    while True:
        
        for i, param in enumerate(params):
            
            best_move = -1
            best_score = -9999999
            
            # do 5 steps per param
            # this is relatively inefficient with an expensive cost function
            # since we evaluate '0' parameter each time, which doesn't need to be evaluated...
            for j, move in enumerate(movements):
                params[i] += params_step[i] * move
                
                # check if move is out of bounds!
                if params[i] < parameter_ranges[i][0] or params[i] > parameter_ranges[i][1]:
                    # then skip, can always revert to '0' or no change
                    params[i] -= params_step[i] * move
                    continue
                
                new_score = score_function(params)
                params[i] -= params_step[i] * move
                
                if new_score > best_score:
                    best_score = new_score
                    best_move = j
            
            if movements[best_move] == 0:
                params_step[i] /= acceleration
            else:
                params[i] += params_step[i] * movements[best_move]
                params_step[i] = params_step[i] * movements[best_move] # accelerate step size
           
        updated_score = score_function(params)
        if updated_score - score < finish_epsilon:
            return (score, params)
        score = updated_score
            


In [5]:
# Hill climbing with random restarts

def random_restart_hillclimb(score_function, parameter_ranges, finish_epsilon=0.1, parameter_step_fraction=0.01, acceleration=1.2, num_restarts=100, initialization_generator=None, append_to=None):
    
    if append_to is None:
        local_score_maxima = []
        local_maxima_params = []
    else:
        local_score_maxima = append_to[0]
        local_maxima_params = append_to[1]
    
    for _ in range(num_restarts):
        if initialization_generator is not None:
            init_params = initialization_generator(parameter_ranges)
        else:
            init_params = None
        
        try:
            score, params = hillclimb(score_function, parameter_ranges, parameter_initializations=init_params, finish_epsilon=finish_epsilon, parameter_step_fraction=parameter_step_fraction, acceleration=acceleration)
            print("{0}, params: {1}".format(score, params))

        except Exception as e: # something...
            print("error!", e)
        
        local_score_maxima.append(score)
        local_maxima_params.append(params)
    return (local_score_maxima, local_maxima_params)
        

In [16]:
road_prob_dist = NormalProbabilityDist(two_stddev=0.4)

camera_height = 6.0
resolution = (50, 50)
yaw_range = 90 # degrees +- from center yaw position

curvature = 0.0
length = 40.0 # does matter! can see end of road!

path = Path()
path.add_segment(curvature, length)
road = Road(path=path, probability_distribution=road_prob_dist, width=3.0, side_offset=1.5)

# place camera halfway along road
allowed_positions_orientations = road.get_allowed_camera_positions_at_distance(length/2.0)
lhs_pos, lhs_direction = allowed_positions_orientations[0]
rhs_pos, rhs_direction = allowed_positions_orientations[1]

# just stick with lhs for now, symmetric with straight road

camera_position = lhs_pos + np.array([0.0, 0.0, camera_height])
camera_pitch_range = np.array([31.0, 85.0])

# convert lhs_direction into a yaw facing the road perpendicularly
center_yaw_rad = np.arccos(np.dot(np.array([1.0,0,0]), h.normalize(lhs_direction)))
center_yaw_deg = np.rad2deg(center_yaw_rad)
camera_yaw_range = np.array([center_yaw_deg - yaw_range, center_yaw_deg + yaw_range])

fov_h_range = np.array([20.0, 120.0])
fov_v_range = fov_h_range.copy()


In [17]:
print(camera_position)
print(center_yaw_deg)


[ 20.  -3.   6.]
90.0


In [8]:
def score_function(params):
    pitch, yaw, h_fov, v_fov = params
    c = Camera(camera_position, pitch, yaw)
    c.set_resolution(*resolution)
    c.set_fov(h_fov, v_fov)
    
    
    error_bounds = error_radius_across_pixels(c)
    pixel_probabilities = c.get_pixel_probabilities_for_road(road)
    
    target = (1/(1+error_bounds)) * pixel_probabilities
    score = np.sum(target)
    print("--score, params: {0}, {1}".format(score, params))
    return score
    

In [9]:
scores_params = [[],[]]

In [10]:
param_ranges = np.array([
    camera_pitch_range, camera_yaw_range, fov_h_range, fov_v_range
])


random_restart_hillclimb(score_function, param_ranges, finish_epsilon=0.1, parameter_step_fraction=0.1, acceleration=1.5, num_restarts=100000, append_to=scores_params)

--score, params: 0.50860186066, [ 78.78662339  16.47247842  74.35845515  87.0121788 ]
--score, params: 0.493452679388, [ 70.68662339  16.47247842  74.35845515  87.0121788 ]
--score, params: 0.50183202317, [ 75.18662339  16.47247842  74.35845515  87.0121788 ]
--score, params: 0.50860186066, [ 78.78662339  16.47247842  74.35845515  87.0121788 ]
--score, params: 0.515664634836, [ 82.38662339  16.47247842  74.35845515  87.0121788 ]
--score, params: 0.545370474944, [ 82.38662339   4.47247842  74.35845515  87.0121788 ]
--score, params: 0.515664634836, [ 82.38662339  16.47247842  74.35845515  87.0121788 ]
--score, params: 0.437754568618, [ 82.38662339  28.47247842  74.35845515  87.0121788 ]
--score, params: 0.427350237761, [ 82.38662339  43.47247842  74.35845515  87.0121788 ]
--score, params: 0.591899775936, [ 82.38662339   4.47247842  59.35845515  87.0121788 ]
--score, params: 0.613967289928, [ 82.38662339   4.47247842  67.69178848  87.0121788 ]
--score, params: 0.545370474944, [ 82.38662339

--score, params: 3.16807351402e-06, [  31.79752865  101.09411465   34.97899453   45.04635584]
--score, params: 0.571444590365, [  31.79752865  101.09411465   34.97899453   78.79635584]
0.558059603434, params: [  31.79752865  101.09411465   34.97899453   78.79635584]
--score, params: 0.389362892694, [  54.6378001    47.37565547  104.32691436   24.38146388]
--score, params: 0.346908889708, [  46.5378001    47.37565547  104.32691436   24.38146388]
--score, params: 0.37140349648, [  51.0378001    47.37565547  104.32691436   24.38146388]
--score, params: 0.389362892694, [  54.6378001    47.37565547  104.32691436   24.38146388]
--score, params: 0.405789323076, [  58.2378001    47.37565547  104.32691436   24.38146388]
--score, params: 0.424059031285, [  62.7378001    47.37565547  104.32691436   24.38146388]
--score, params: 0.306351357391, [  62.7378001    20.37565547  104.32691436   24.38146388]
--score, params: 0.352194293064, [  62.7378001    35.37565547  104.32691436   24.38146388]
--scor

--score, params: 0.600316026517, [  75.88147556  134.90363318   45.10092628   68.80966945]
--score, params: 0.560530125435, [  75.88147556  134.90363318   53.43425961   68.80966945]
--score, params: 0.732385688826, [  75.88147556  134.90363318   23.43425961   53.80966945]
--score, params: 0.750189286882, [  75.88147556  134.90363318   23.43425961   62.14300278]
--score, params: 0.669345008324, [  75.88147556  134.90363318   23.43425961   68.80966945]
--score, params: 0.59228825769, [  75.88147556  134.90363318   23.43425961   75.47633612]
--score, params: 0.510810501996, [  75.88147556  134.90363318   23.43425961   83.80966945]
--score, params: 0.750189286882, [  75.88147556  134.90363318   23.43425961   62.14300278]
--score, params: 0.616495806831, [  81.28147556  134.90363318   23.43425961   62.14300278]
--score, params: 0.716374686496, [  78.28147556  134.90363318   23.43425961   62.14300278]
--score, params: 0.750189286882, [  75.88147556  134.90363318   23.43425961   62.14300278]


--score, params: 0.61924023566, [ 81.50904061  84.12287524  22.8224594   62.59233163]
--score, params: 0.639432713356, [  81.50904061  106.62287524   22.8224594    62.59233163]
--score, params: 0.723450834163, [  81.50904061  124.62287524   22.8224594    62.59233163]
--score, params: 0.466083379589, [  81.50904061  142.62287524   22.8224594    62.59233163]
--score, params: 0.000997561898951, [  81.50904061  165.12287524   22.8224594    62.59233163]
--score, params: 0.685042771912, [  81.50904061  124.62287524   32.8224594    62.59233163]
--score, params: 0.71331119025, [  81.50904061  124.62287524   27.26690384   62.59233163]
--score, params: 0.723450834163, [  81.50904061  124.62287524   22.8224594    62.59233163]
--score, params: 0.481664591998, [  81.50904061  124.62287524   22.8224594    85.09233163]
--score, params: 0.601897168464, [  81.50904061  124.62287524   22.8224594    72.59233163]
--score, params: 0.723450834163, [  81.50904061  124.62287524   22.8224594    62.59233163]
--

--score, params: 0.442652234251, [ 51.76972828  33.46321318  83.50315687  69.38503422]
--score, params: 0.536280213915, [ 51.76972828  48.46321318  83.50315687  69.38503422]
--score, params: 0.569129331807, [ 51.76972828  48.46321318  68.50315687  69.38503422]
--score, params: 0.563118928535, [ 51.76972828  48.46321318  76.83649021  69.38503422]
--score, params: 0.536280213915, [ 51.76972828  48.46321318  83.50315687  69.38503422]
--score, params: 0.480742877482, [ 51.76972828  48.46321318  90.16982354  69.38503422]
--score, params: 0.415451351047, [ 51.76972828  48.46321318  98.50315687  69.38503422]
--score, params: 0.695975761669, [ 51.76972828  48.46321318  68.50315687  54.38503422]
--score, params: 0.639793140437, [ 51.76972828  48.46321318  68.50315687  62.71836755]
--score, params: 0.569129331807, [ 51.76972828  48.46321318  68.50315687  69.38503422]
--score, params: 0.503918047308, [ 51.76972828  48.46321318  68.50315687  76.05170089]
--score, params: 0.43472328901, [ 51.769728

...

--score, params: 0.84986389112, [ 62.16150331  37.95978703  38.84338537  45.59487801]
--score, params: 0.60742487908, [ 50.46150331  10.95978703  38.84338537  45.59487801]
--score, params: 0.863254156706, [ 50.46150331  25.95978703  38.84338537  45.59487801]
--score, params: 0.984843488641, [ 50.46150331  37.95978703  38.84338537  45.59487801]
--score, params: 0.93294208398, [ 50.46150331  49.95978703  38.84338537  45.59487801]
--score, params: 0.88315874084, [ 50.46150331  64.95978703  38.84338537  45.59487801]
--score, params: 0.988076766095, [ 50.46150331  37.95978703  23.84338537  45.59487801]
--score, params: 0.987997766726, [ 50.46150331  37.95978703  32.1767187   45.59487801]
--score, params: 0.984843488641, [ 50.46150331  37.95978703  38.84338537  45.59487801]
--score, params: 0.946354292197, [ 50.46150331  37.95978703  45.51005204  45.59487801]
--score, params: 0.804507471156, [ 50.46150331  37.95978703  53.84338537  45.59487801]
--score, params: 1.51167624968, [ 50.46150331  

--score, params: 0.372207450115, [ 79.71771989  10.74047036  45.08982291  91.2413866 ]
--score, params: 0.349632370549, [ 83.31771989  10.74047036  45.08982291  91.2413866 ]
--score, params: 0.396939822856, [ 71.61771989  10.74047036  45.08982291  91.2413866 ]
--score, params: 0.457962110665, [ 71.61771989  22.74047036  45.08982291  91.2413866 ]
--score, params: 0.465134969628, [ 71.61771989  37.74047036  45.08982291  91.2413866 ]
--score, params: 0.465134768308, [ 71.61771989  37.74047036  30.08982291  91.2413866 ]
--score, params: 0.465135708975, [ 71.61771989  37.74047036  38.42315624  91.2413866 ]
--score, params: 0.465134969628, [ 71.61771989  37.74047036  45.08982291  91.2413866 ]
--score, params: 0.465137448454, [ 71.61771989  37.74047036  51.75648958  91.2413866 ]
--score, params: 0.460588452486, [ 71.61771989  37.74047036  60.08982291  91.2413866 ]
--score, params: 0.539109908738, [ 71.61771989  37.74047036  51.75648958  76.2413866 ]
--score, params: 0.513521713268, [ 71.61771

--score, params: 2.02876991076, [ 52.71771989  43.07380369  20.08982291  22.07471994]
--score, params: 1.88830009224, [ 50.31771989  43.07380369  20.08982291  22.07471994]
--score, params: 1.56203623852, [ 47.31771989  43.07380369  20.08982291  22.07471994]
--score, params: 1.6262559795, [ 55.11771989  35.07380369  20.08982291  22.07471994]
--score, params: 1.90497688467, [ 55.11771989  39.51824814  20.08982291  22.07471994]
--score, params: 2.03455004535, [ 55.11771989  43.07380369  20.08982291  22.07471994]
--score, params: 2.01896138446, [ 55.11771989  46.62935925  20.08982291  22.07471994]
--score, params: 1.95931095565, [ 55.11771989  51.07380369  20.08982291  22.07471994]
--score, params: 1.69658671155, [ 55.11771989  43.07380369  30.08982291  22.07471994]
--score, params: 1.92343014317, [ 55.11771989  43.07380369  24.53426735  22.07471994]
--score, params: 2.03455004535, [ 55.11771989  43.07380369  20.08982291  22.07471994]
--score, params: 1.40670611758, [ 55.11771989  43.07380

--score, params: 0.303596541906, [  68.20288746   27.66321686   46.97982673  119.21109765]
--score, params: 0.40261871444, [  68.20288746   27.66321686   46.97982673  104.21109765]
--score, params: 0.343005472756, [  56.05288746   27.66321686   46.97982673  104.21109765]
--score, params: 0.372323372242, [  62.80288746   27.66321686   46.97982673  104.21109765]
--score, params: 0.40261871444, [  68.20288746   27.66321686   46.97982673  104.21109765]
--score, params: 0.441452449883, [  73.60288746   27.66321686   46.97982673  104.21109765]
--score, params: 0.448342161151, [  80.35288746   27.66321686   46.97982673  104.21109765]
--score, params: 0.306491626066, [  80.35288746   68.16321686   46.97982673  104.21109765]
--score, params: 0.378675194063, [  80.35288746   45.66321686   46.97982673  104.21109765]
--score, params: 0.448342161151, [  80.35288746   27.66321686   46.97982673  104.21109765]
--score, params: 0.378382872168, [  80.35288746    9.66321686   46.97982673  104.21109765]
-

KeyboardInterrupt: 

In [11]:
print(len(scores_params))
print(len(scores_params[0]), len(scores_params[1]))
hillclimb_results_scores = np.array(scores_params[0])
hillclimb_results_params = np.array(scores_params[1])
sorted_indices = np.argsort(hillclimb_results_scores)

2
(39, 39)


In [12]:
for index in sorted_indices:
    print("{0:.4f}\t ---\t{1}".format(hillclimb_results_scores[index], hillclimb_results_params[index]))


0.3206	 ---	[ 84.53945406  37.1926512   70.57933697  91.31700453]
0.4779	 ---	[  76.60433001   25.15698458   27.66709305  100.0154367 ]
0.4850	 ---	[  80.55056084  162.30752459   75.52714607   58.13641696]
0.5038	 ---	[ 84.46771074  15.89245559  72.25734342  71.18858372]
0.5581	 ---	[  31.79752865  101.09411465   34.97899453   78.79635584]
0.5639	 ---	[  78.23309705  173.13090479   63.32160445   54.59975107]
0.5827	 ---	[  80.90099333  133.43402059   86.37068483   27.7890546 ]
0.6132	 ---	[  79.04264005  137.58824486   28.49993102   72.27936879]
0.6161	 ---	[ 84.78662339   4.47247842  63.24734404  49.5121788 ]
0.6363	 ---	[  35.1322019   100.89065674   21.90921437   64.66598637]
0.6367	 ---	[  83.3134128   157.45794769   66.29374887   27.38577091]
0.6374	 ---	[  81.4131853   175.12239152   64.80861428   35.20095621]
0.7043	 ---	[  81.89349614  140.83204641   68.40215375   22.7406527 ]
0.7235	 ---	[  81.50904061  124.62287524   22.8224594    62.59233163]
0.7502	 ---	[  75.88147556  134.

In [13]:
np.save("data/hillclimbing/0_curve_pitch_yaw_fov_6m_scores.npy", hillclimb_results_scores[sorted_indices])
np.save("data/hillclimbing/0_curve_pitch_yaw_fov_6m_params.npy", hillclimb_results_params[sorted_indices])


## Also allow optimizing for height!

In [28]:
c2 = Camera(camera_position, 56.3126941, 46.40148737)
c2.set_fov(20.01860597, 20.27228136)
c2.set_resolution(200,200)

In [29]:
pixel_probabilities = c2.get_pixel_probabilities_for_road(road)

In [30]:
print(np.max(pixel_probabilities))

0.00398935258458


In [ ]:
road_prob_dist = NormalProbabilityDist(two_stddev=0.4)

camera_height_
resolution = (100, 100)
yaw_range = 60 # degrees +- from center yaw position

curvature = 0.0
length = 40.0 # doesn't really matter

path = Path()
path.add_segment(curvature, length)
road = Road(path=path, probability_distribution=road_prob_dist, width=3.0, side_offset=1.5)

# place camera halfway along road
allowed_positions_orientations = road.get_allowed_camera_positions_at_distance(length/2.0)
lhs_pos, lhs_direction = allowed_positions_orientations[0]
rhs_pos, rhs_direction = allowed_positions_orientations[1]

# just stick with lhs for now, symmetric with straight road

camera_position = lhs_pos + np.array([0.0, 0.0, camera_height])
camera_pitch_range = np.array([31.0, 85.0])

# convert lhs_direction into a yaw facing the road perpendicularly
center_yaw_rad = np.arccos(np.dot(np.array([1.0,0,0]), h.normalize(lhs_direction)))
center_yaw_deg = np.rad2deg(center_yaw_rad)
camera_yaw_range = np.array([center_yaw_deg - yaw_range, center_yaw_deg + yaw_range])

fov_h_range = np.array([20.0, 120.0])
fov_v_range = fov_h_range.copy()
